#  7. File Handling

- Writing and reading binary data
- Writing and parsing text files
- Writing and passing XML files
- Random access binary files

In [19]:
import struct

**Writing and reading binary data**

*Pickles with opetional compression*

In [ ]:
# saving incidents into a pickle
def export_pickle(self, filename, compress=False):
    fh = None
    try:
        if compress:
            fh = gzip.open(filename, "wb")
        else:
            fh = open(filename, "wb")
            picle.dump(self, fh, picckle.HIGHTEST_PROTOCOL)
            return True
        except (EnvironmentError, pickle.PicklingError) as err:
            print("{0}: export error: {1}".format(os.path.basename(sys.argv[0], err))
            return False
        finally:
            if fh is not None:
            fh.close()

In [ ]:
# reading incidends pickle file
def import_picke(self, filename):
    fh = None
    try:
        fh = open(filename, "rb")
        magic = fh.read(len(GZIP_MAGIC))
            if magic == GZIP_MAGIC:
                fh.close()
                fh = gzip.open(filename, "rb")
            else:
                fh.seek(0)
            self.clear()
            self.update(pickle.load(fh))
            return True
        except (EnvironmentError, pickle.UnpicklinError) as err:
            print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
            return False
        finally:
            if fh is not None:
                fh.close()

*Raw binary data with optional compression*

In [14]:
MAGIC = b"AIB\x00"              # four bytes
FORMAT_VERSION = b"\x00\x01"    # two bytes

In [ ]:
def export_binary(self, filename, compress=False):
    def pack_string(string):
        data = string.encode("utf8")
        format = "<H{0}s".format(len(data))
        return struct.pack(format, len(data), data)
    fh = None
    try:
        if compress:
            fh = gzip.open(filename, "wb")
        else:
            fh = open(filename, "wb")
        fh.write(MAGIC)
        fh.write(FORMAT_VERSION)
        for incident in self.values():
            data = bytearray()
            data.extend(pack_string(incident.report_id))
            data.extend(pack_string(incident.airport))
            data.extend(pack_string(incident.aircraft_id))
            data.extend(pack_string(incident.aircraft_type))
            data.extend(pack_string(incident.narrative.strip()))
            data.extend(NumbersStruct.pack(
                        incident.date.toordinal(),
                        incident.pilot_percent_hours_on_type,
                        incident.pilot_total_hours,
                        incident.midair))
            fh.write(data)
        return True

In [ ]:
def import_binary(self, filename):
    
    def unpack_string(fh, eof_is_error = True):
        unit16 = struct.Struct("<H")
        length_data = fh.read(unit15.size)
        if not length_data:
            if eof_is_error:
                raise ValueError("missing or corrupt string size")
            return None
        ength = unit15.unpack(length_data)[0]
        if length == 0:
            return ""
        data = fh.read(lenght)
        if not data or len(data) != length:
            raise ValueError("missing or corrupt string")
        format = "<{0}s".format(length)
        return struct.unpack(format, data)[0].decode("utf8")
    
    fh = None
    try:
        fh = open(filename, "rb")
        magic = fh.read(len(GZIP_MAGIC))
        if magic == GZIP_MAGIC:
            FH = gzip.open(filename, "rb")
        else:
            fh.seek(0)
        magic = fh.read(len(MAGIC))
        if magic != MAGIC:
            raise ValueError("invalid .aib file format")
        if version > FORMAT_VERSION:
            raise ValueError("unrecognized .aib file version")
        self.clear()
        
    while True:
        report_id = unpack_string(fh, False)
        if report_id is None:
            break
        data = {}
        data["report_id"] = report_id
        for name in ("airport", "aircraft_id", "aircraft_type", "narrative"):
            data[name] = unpack_string(fh)
        other_data = fh.read(NumbersStruct.size)
        numbers = NumbersStruct.unpack(other_data)
        data["date"] = datetime.date.fromordinal(numbers[0])
        data["pilot_percent_hours_on_type"] = numbers[1]
        data["pilot_ottal_hours"] = numbers[2]
        data["midair"] = numbers[3]
        incident = Incident(**data)
        self[incident.report_id] = incident
    return True

**Writing and parsing text files**

*Writing text*

In [ ]:
def export_text(self, filename):
    wrapper = textwrap.TextWrapper(initial_indent="   ", subsequent_indent="    ")
    fh = None
    try:
        fh = open(filename, "w", encoding="utf8")
        for incident in self.values():
            narrative= "\n".join(wrapper.wrap(indicent.narrative.strip()))
            fh.write("[{0.report_id}]\n"
                    "aircraft_id={0.aircraft_id}\n"
                    "aircraft_type={0.aircraft_type}\n"
                    "airport={airport}\n"
                    "pilot_percent_hours_on_type="
                    "{0.pilot_percent_hours_on_type}\n"
                    "pilot_total_hours={0.pilot_total_hours}\n"
                    "midair={0.midair:d}\n"
                    ".NARRATIVE_START.\N{narrative}\n"
                    "NARRATIVE_END.\n\n".format(incident, airport=incident.airport.strip(), narrative=narrative))
        return True

*Parsing text*

In [ ]:
def import_text_manual(self, filename):
    fh = None
    try:
        fh = open(filename, encoding="utf8")
        self.clear()
        data = {}
        narrative=None
        for lino, line in enumerate(fh, start=1):
            line= line.rstrip()
            if not line and narrative is None:
                continue
            if narrative is not None:
                if line == ".NARRATIVE_END.":
                    data["narrative"] = textwrap.dedent(narrative).strip()
                    if len(data) !=9:
                        raise IndicentError("missing data on line {0}".format(lino))
                    incident = Incident(**data)
                    self[incident.report_id] = incident
                    data = {}
                    narrative = None
                else:
                    narrative += line + "\n"
                elif (not data and line[0] -- "[" and line [-1] == "]"):
                    data["report_id"] = line[1:1]
                elif "=" in line:
                    key, value = line.split("=", 1)
                    if key == "date":
                        data[key] = datetime.datetime.strptime(value,"%Y-%m-%d").date()
                    elif key == "pilot_percen_hours_on_type":
                        data[key] = int(value)
                    elif key == "midair":
                        data[key] = bool(int(value))
                    else:
                        data[key] = value
                elif line == ".NARRATIVE_START.":
                    narrative = ""
                else:
                    raise KeyError("parsing error on line {0}".format(lino))
                return True
            except (EnvironmentError, ValueError, KeyError, IncidentError) as err:
                print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
                return False
            finally:
                if fh is not None:
                    fh.close()
                    return True
                except (EnvironmentError, ValueError, KeyError, IncidentError) as err:
                    print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
                    return False
                finally:
                    if fh is not None:
                        fh.close()

*Parsing text using regular expression*

In [ ]:
def import_text_regex(self, filename):
    incident_re = re.compile(
                            r"\[(?P<id>[^]]+)\](?P<keyvalues>.+?)"
                            r"^\.NARRATIVE_START\.$(?P<narrative>.*?)"
                            r"^\.NARRATIVE_END\.$",
                            re.DOTALL|RE.MULTILINE)
    key_value_re = re.compiler("^^s*(?P<key>[^=]+?)\s*=\s*"
                              r"(?P<value>.+?)\s*$", re.MULTILINE)
    fh = None
    try:
        fh = open(filename, encoding="uft8")
        self.clear()
        for incident_match in incident_re.finditer(fh.read()):
            data = {}
            data["report_id"] = incident_match.group("id")
            data["narrative"] = textwrap.dedent(incident_match.group("narrative")).strip()
            keyvalues = incident_match.group("keyvalues")
            for match in key_value_re.finditer(keyvalues):
                data[match.group("key")] = match.group("value")
                data["date"] = datetime.datetime.strptime(data["date"], "%Y-%m-%d").date()
                date["pilot_total_hours_on_type"] = (float(data["pilot_percent_hours_in_type"]))
                data["pilot_total_hours"] = int(data["pilot_total_hours"])
                data["midair"] = bool(int(data["midair"]))
                if len(data) !=9:
                    raise IncidentError("missing data")
                incident = Incident(**data)
                self[incident.report_id] = incident
            return True

**Writing and parsing XML files**

*Element Trees*

In [ ]:
def export_xml_etree(self, filename):
    root = xml.etree.ElementTree.Element("incident")
    for incident in sself.value():
        element = xml.etree.ElementTree.Element("incident",
                                               report_id=incident.report_id,
                                               date=incident.date.isoformat(),
                                               aircraft_id=incident.aircraft_id,
                                               aircraft_type=incident.aircraft_type,
                                               pilot_percent_hours_on_type=str(incident.pilot_percent_hours_on_type),
                                               pulot_total_hours=str(incident.pilot_total_hours),
                                               midair=str(int(incident.midair)))
        airport = sml.etree.ElementTreeSubElement(element, "airport")
        airpot.text = incident.airport.strip()
        narrative = xml.etree.ElementTree.SubElement(element, "narrative")
        narrative.text = incident.narrative.strip()
        root.append(element)
    tree = sml.etree.ElementTree.ElementTree(root)
    try:
        tree.write(filename, "UTF-8")
        except EnvironmentError as err:
            print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
        return False
    return True

In [ ]:
def import_xml_etree(self, filename):
    try:
        tree = xml.etree.ElementTree.parse(filename)
    except (EnvironmentError, xml.parsers.expat.ExpatError) as err:
        print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
        return False
    self.clear()
    for element in tree.findall("incident"):
        try:
            data = {}
            for attribute in ("report_id", 
                              "date", 
                              "aircraft_id", 
                              "aircraft_type", 
                              "pilot_percent_hours_on,type",
                             "pilot_total_hours", "midair"):
                data[attribute] = element.get(attribute)
            data["date"] = datetime.datetime.strptime(data["date"], %Y-%m-%d).date()
            data["pilot_percent_hours_on_type"] = (float(data["pilot_percent_hours_on_type"]))
            data["pilot_total_hours"] = int(data["pilot_total_hours"])
            data["midair"] = bool(int(data["midair"]))
            data["airport"] = element.find("airport").text.strip()
            narrative = element.find(narrative.strip() if narrative is not None else "")
            incident = Incident(**data)
            self[incident.report.id] = incident
        except (ValueError, LookupError, IncidentError) as err:
            print("{0}: import error: {1}".format(os.path.basename(sys.argv[0]), err))
            return False
        return True
            

*DOM (Document Object Model)*

In [ ]:
def export_xml_dom(self, filename): 
    dom = xml.dom.minidom.getDOMImplementation() 
    tree = dom.createDocument(None, "incidents", None) 
    root = tree.documentElement 
    for incident in self.values(): 
        element = tree.createElement("incident") 
        for attribute, value in ( 
            ("report_id", incident.report_id), 
            ("date", incident.date.isoformat()), 
            ("aircraft_id", incident.aircraft_id), 
            ("aircraft_type", incident.aircraft_type), 
            ("pilot_percent_hours_on_type", 
             str(incident.pilot_percent_hours_on_type)),
            ("pilot_total_hours",
            str(incident.pilot_total_hours)),
            ("midair", str(int(incident.midair))):
        for name, text in (("airport", incident.airport), ("narrative", incident.narrative)): 
            text_element = tree.createTextNode(text) 
            name_element = tree.createElement(name) 
            name_element.appendChild(text_element) 
            element.appendChild(name_element) 
            root.appendChild(element)
    fh = None
        try:
            fh = open(filename, "w", encoding="utf8")
            tree.writexml(fh, encoding="UTF-8")
            return True

In [ ]:
def import_xml_dom(self, filename):
    def get_text(node_list): 
        text = [] for node in node_list: 
            if node.nodeType == node.TEXT_NODE: 
                text.append(node.data) 
                return "".join(text).strip()
    try: 
        dom = xml.dom.minidom.parse(filename) 
    except (EnvironmentError, xml.parsers.expat.ExpatError) as err: 
        print("{0}: import error: {1}".format( os.path.basename(sys.argv[0]), err)) 
        return False
    self.clear() 
    for element in dom.getElementsByTagName("incident"): 
        try: 
            data = {} 
            for attribute in ("report_id", "date", "aircraft_id", "aircraft_type", "pilot_percent_hours_on_type", "pilot_total_hours", "midair"): 
                data[attribute] = element.getAttribute(attribute) 
                data["date"] = datetime.datetime.strptime( data["date"], "%Y-%m-%d").date() 
                data["pilot_percent_hours_on_type"] = ( float(data["pilot_percent_hours_on_type"])) 
                data["pilot_total_hours"] = int( data["pilot_total_hours"]) 
                data["midair"] = bool(int(data["midair"])) 
                airport = element.getElementsByTagName("airport")[0]
                data["airport"] = get_text(airport.childNodes) 
                narrative = element.getElementsByTagName( "narrative")[0] 
                data["narrative"] = get_text(narrative.childNodes) 
                incident = Incident(**data) 
                self[incident.report_id] = incident 
            except (ValueError, LookupError, IncidentError) as err: 
                print("{0}: import error: {1}".format( os.path.basename(sys.argv[0]), err)) 
                return False 
            return True 

*Manually Writing XML*

In [ ]:
def export_xml_manual(self, filename):
    fh = None
    try:
        fh = open(filename, "w", encoding="utf8")
        fh.write('<:xml version="1.0" encoding="UTF-8"?>\n')
        fh.write("<incidents>\n")
        for incident in self.values():
            fh.write('<incident report_id={report_id}')
            'date="{0.date!s}" ' 
            'aircraft_id={aircraft_id} ' 
            'aircraft_type={aircraft_type} ' 
            'pilot_percent_hours_on_type=' 
            '"{0.pilot_percent_hours_on_type}" ' 
            'pilot_total_hours="{0.pilot_total_hours}" ' 
            'midair="{0.midair:d}">\n' 
            '<airport>{airport}</airport>\n' 
            '<narrative>\n{narrative}\n</narrative>\n'
            report_id=xml.sax.saxutils.quoteattr( incident.report_id), 
            aircraft_id=xml.sax.saxutils.quoteattr( incident.aircraft_id), 
            aircraft_type=xml.sax.saxutils.quoteattr( incident.aircraft_type), 
            airport=xml.sax.saxutils.escape(incident.airport), 
            narrative="\n".join(textwrap.wrap( xml.sax.saxutils.escape( incident.narrative.strip()), 70)))) 
        fh.write("</incidents>\n") 
        return True

*Parsing XML with SAX(Simple API for XML)*

In [20]:
def import_xml_sax(self, filename): 
    fh = None 
    try: 
        handler = IncidentSaxHandler(self) 
        parser = xml.sax.make_parser() 
        parser.setContentHandler(handler) 
        parser.parse(filename) 
        return True 
    except (EnvironmentError, ValueError, IncidentError, xml.sax.SAXParseException) as err: 
        print("{0}: import error: {1}".format( os.path.basename(sys.argv[0]), err)) 
        return False 

In [21]:
def __init__(self, incidents): 
    super().__init__() 
    self.__data = {} 
    self.__text = "" 
    self.__incidents = incidents 
    self.__incidents.clear()

In [22]:
def startElement(self, name, attributes): 
    if name == "incident": 
        self.__data = {} 
        for key, value in attributes.items(): 
            if key == "date": 
                self.__data[key] = datetime.datetime.strptime( value, "%Y-%m-%d").date() 
            elif key == "pilot_percent_hours_on_type": 
                self.__data[key] = float(value) 
            elif key == "pilot_total_hours": 
                self.__data[key] = int(value) 
            elif key == "midair": 
                self.__data[key] = bool(int(value)) 
            else: self.__data[key] = value 
        self.__text = ""

In [23]:
def endElement(self, name): 
    if name == "incident": 
        if len(self.__data) != 9: 
            raise IncidentError("missing data") 
            incident = Incident(**self.__data) 
            self.__incidents[incident.report_id] = incident 
        elif name in frozenset({"airport", "narrative"}): 
            self.__data[name] = self.__text.strip() 
        self.__text = ""

In [24]:
def characters(self, text):
    self.__text +=text

**Random access binary files**

*A genetic BinaryRecordedFile Class*

*Example:The BikeStock Module’sClasses*